In [ ]:
!pip install selenium

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

# Selenium setup
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=chrome_options)

# Data lists
product = []
price = []
brands = []
images = []
productlink = []

# Function to process a single page
def process_page(page_no):
    url = f"https://www.flipkart.com/clothing-and-accessories/ethnic-wear/kurtas/women-kurtas-and-kurtis/pr?sid=clo%2Ccfv%2Ccib%2Crkt&q=kurtas+kurtis&otracker=categorytree&otracker=nmenu_sub_Women_0_Kurtas+%26+Kurtis&fm=neo%2Fmerchandising&iid=M_737202f5-e74d-4785-9ee1-6b05e929bad3_1_372UD5BXDFYS_MC.0VYNO3UZWA0J&otracker=hp_rich_navigation_3_1.navigationCard.RICH_NAVIGATION_Fashion%7EWomen%2BEthnic%7EWomen%2BKurtas%2B%2526%2BKurtis_0VYNO3UZWA0J&otracker1=hp_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_3_L2_view-all&cid=0VYNO3UZWA0J&page={page_no}"
    driver.get(url)
    time.sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    entries = soup.find_all('div', '_1sdMkc LFEi7Z')
    if entries:
        for entry in entries:
            # Getting the product name
            try:
                pr = entry.find("a", attrs={"class": "WKTcLC"}).get("title")
                product.append(pr)
            except Exception as e:
                product.append(None)

            # Getting the price
            try:
                pri = entry.find("div", attrs={"class": "Nx9bqj"}).text
                price.append(pri)
            except Exception as e:
                price.append(None)

            # Getting the image
            try:
                img = entry.find("img", attrs={"class": "_53J4C-"}).get("src")
                images.append(img)
            except Exception as e:
                images.append(None)

            # Getting the product link
            try:
                link = entry.find("a", attrs={"class": "rPDeLR"}).get("href")
                productlink.append("https://www.flipkart.com" + link)
            except Exception as e:
                productlink.append(None)

            # Getting the brand
            try:
                br = entry.find("div", attrs={"class": "syl9yP"}).text
                brands.append(br)
            except Exception as e:
                brands.append(None)
    else:
        print("No entries found with the specified class on page", page_no)
        print("Here's some of the page content to help you debug:")
        print(soup.prettify()[:100])

for page_no in range(1, 26):
    process_page(page_no)

# Close the Selenium driver
driver.quit()

# Creating DataFrame and saving to CSV
df = pd.DataFrame({'Image': images, 'Brand': brands, 'Name': product, 'Price': price, 'Productlink':productlink})
print(df)
df.to_csv('flipkart_kurtas.csv', index=False)

In [ ]:
len(productlink)

1000

In [ ]:
df=pd.read_csv("flipkart_kurtas.csv")
productlink=list(df["Productlink"])


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# Selenium setup
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=chrome_options)

rating = []
sizechart = []

def process_product_link(link, i):
    try:
        driver.get(link)
        time.sleep(5)

        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        # Getting the rating
        try:
            rate = soup.find("div", class_="XQDdHH _1Quie7").text
            rating.append(rate)
        except Exception as e:
            print(f"Failed to get rating for {i}: {e}")
            rating.append(None)

        # Getting the size chart
        try:
            size_chart = soup.find("img", class_="CDkImq").get("src")
            sizechart.append(size_chart)
        except Exception as e:
            print(f"Failed to get size chart for {i}: {e}")
            sizechart.append(None)

    except Exception as e:
        print(f"Failed to process link for {i}: {e}")
        rating.append(None)
        sizechart.append(None)


for i, link in enumerate(productlink):
    process_product_link(link, i)

# Close the Selenium driver
driver.quit()


# Creating DataFrame and saving to CSV
df1 = pd.DataFrame({'Rating': rating, 'Size Chart': sizechart})
print(df1)
df1.to_csv('flipkart_kurtas1.csv', index=False)


In [ ]:

# Selenium setup
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=chrome_options)
sizechart = []

def process_product_link(link, i):
    try:
        driver.get(link)
        time.sleep(5)  

        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.XQDdHH._1Quie7")))

        size_chart_link = driver.find_element(By.CSS_SELECTOR, "div.mNjYnk")
        size_chart_link.click()

        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.hRtgfi")))

        size_chart_html = driver.find_element(By.CSS_SELECTOR, "div.hRtgfi").get_attribute("outerHTML")

        
        soup = BeautifulSoup(size_chart_html, 'html.parser')
        table = soup.find('table')

        if table:
            # Extracting table rows
            rows = table.find_all('tr')
            size_chart_data = []

            for row in rows:
                columns = row.find_all('td')
                row_data = [column.get_text(strip=True) for column in columns]
                size_chart_data.append(row_data)

            sizechart.append(size_chart_data)
        else:
            print(f"No size chart table found for {i}")
            sizechart.append(None)

    except Exception as e:
        print(f"Failed to get size chart for {i}")
        sizechart.append(None)

for i, link in enumerate(productlink):
    if link:
        process_product_link(link, i)
    else:
        sizechart.append(None)

# Close the Selenium driver
driver.quit()

# Creating DataFrame and saving to CSV
df3 = pd.DataFrame({'Size Chart': sizechart})
print(df3)
df3.to_csv('flipkart_kurtas1.csv', index=False)
